# How to delete messages

One of the common states for a graph is a list of messages. Usually you only add messages to that state. However, sometimes you may want to remove messages (either by directly modifying the state or as part of the graph). To do that, you can use the `RemoveMessage` modifier. In this guide, we will cover how to do that.

The key idea is that each state key has a `reducer` key. This key specifies how to combine updates to the state. The default `MessagesState` has a messages key, and the reducer for that key accepts these `RemoveMessage` modifiers. That reducer then uses these `RemoveMessage` to delete messages from the key.

So note that just because your graph state has a key that is a list of messages, it doesn't mean that that this `RemoveMessage` modifier will work. You also have to have a `reducer` defined that knows how to work with this.

**NOTE**: Many models expect certain rules around lists of messages. For example, some expect them to start with a `user` message, others expect all messages with tool calls to be followed by a tool message. **When deleting messages, you will want to make sure you don't violate these rules.**

## Setup

First, let's build a simple graph that uses messages. Note that it's using the `MessagesState` which has the required `reducer`.

In [2]:
# %%capture --no-stderr
# %pip install --quiet -U langgraph langchain_anthropic

Next, we need to set API keys for Anthropic (the LLM we will use)

In [3]:
import logging
from dotenv import load_dotenv

from IPython.display import Image, display

Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

In [4]:
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

load_dotenv()

True

## Build the agent
Let's now build a simple ReAct style agent.

In [5]:
from typing import Literal

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import ToolNode

memory = SqliteSaver.from_conn_string(":memory:")


@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    return [
        "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
    ]


tools = [search]
tool_node = ToolNode(tools)
model = ChatOpenAI(model="gpt-4o-mini")
bound_model = model.bind_tools(tools)


def should_continue(state: MessagesState) -> Literal["action", "__end__"]:
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "__end__"
    # Otherwise if there is, we continue
    return "action"


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)

In [6]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="hi! I'm Tam")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


input_message = HumanMessage(content="what's my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm Tam


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Hi, Tam! How can I assist you today?
================================ Human Message =================================

what's my name?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Your name is Tam! How can I help you today?


## Manually deleting messages

First, we will cover how to manually delete messages. Let's take a look at the current state of the thread:

In [7]:
messages = app.get_state(config).values['messages']
messages

[HumanMessage(content="hi! I'm Tam", id='f44d5cb8-ef85-4b9e-b6e6-9989aef37c2c'),
 AIMessage(content='Hi, Tam! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7dd529cfca', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ad7228e-46cc-441f-931a-59fa3773e140-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22}),
 HumanMessage(content="what's my name?", id='527004f6-b0dc-496f-b4fc-2d5b84a56fc2'),
 AIMessage(content='Your name is Tam! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 35, 'total_tokens': 47}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8b761cb050', 'finish_reason': 'stop', 'logprobs': None}, id='run-ec123e1d-edac-4607-bd3e-f2ada6f489e5-0', usage_metadata={'input_tokens': 35, 'output_tokens': 12, 'total_tokens': 47})]

We can call `update_state` and pass in the id of the first message. This will delete that message.

In [8]:
from langchain_core.messages import RemoveMessage
app.update_state(config, {"messages": RemoveMessage(id=messages[0].id)})

/Users/may/.virtualenvs/generative_ai/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The class `RemoveMessage` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


{'configurable': {'thread_id': '2',
  'thread_ts': '1ef45e3e-f9b6-677a-8005-b562cc8db4b8'}}

If we now look at the messages, we can verify that the first one was deleted.

In [9]:
messages = app.get_state(config).values['messages']
messages

[AIMessage(content='Hi, Tam! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7dd529cfca', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ad7228e-46cc-441f-931a-59fa3773e140-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22}),
 HumanMessage(content="what's my name?", id='527004f6-b0dc-496f-b4fc-2d5b84a56fc2'),
 AIMessage(content='Your name is Tam! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 35, 'total_tokens': 47}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8b761cb050', 'finish_reason': 'stop', 'logprobs': None}, id='run-ec123e1d-edac-4607-bd3e-f2ada6f489e5-0', usage_metadata={'input_tokens': 35, 'output_tokens': 12, 'total_tokens': 47})]

## Programmatically deleting messages

We can also delete messages programmatically from inside the graph. Here we'll modify the graph to delete any old messages (longer than 3 messages ago) at the end of a graph run.

In [10]:
from langchain_core.messages import RemoveMessage
from langgraph.graph import END


def delete_messages(state):
    messages = state['messages']
    if len(messages) > 3:
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:-3]]}

# We need to modify the logic to call delete_messages rather than end right away
def should_continue(state: MessagesState) -> Literal["action", "delete_messages"]:
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we call our delete_messages function
    if not last_message.tool_calls:
        return "delete_messages"
    # Otherwise if there is, we continue
    return "action"


# Define a new graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# This is our new node we're defining
workflow.add_node(delete_messages)


workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue,)
workflow.add_edge("action", "agent")

# This is the new edge we're adding: after we delete messages, we finish
workflow.add_edge("delete_messages", END)
app = workflow.compile(checkpointer=memory)

We can now try this out. We can call the graph twice and then check the state

In [11]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "3"}}
input_message = HumanMessage(content="hi! I'm Tam")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])


input_message = HumanMessage(content="what's my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('human', "hi! I'm Tam")]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[('human', "hi! I'm Tam"), ('ai', 'Hi Tam! How can I assist you today?')]
[('human', "hi! I'm Tam"), ('ai', 'Hi Tam! How can I assist you today?'), ('human', "what's my name?")]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[('human', "hi! I'm Tam"), ('ai', 'Hi Tam! How can I assist you today?'), ('human', "what's my name?"), ('ai', 'Your name is Tam! How can I help you today?')]
[('ai', 'Hi Tam! How can I assist you today?'), ('human', "what's my name?"), ('ai', 'Your name is Tam! How can I help you today?')]


If we now check the state, we should see that it is only three messages long. This is because we just deleted the earlier messages - otherwise it would be four!

In [12]:
messages = app.get_state(config).values['messages']
messages

[AIMessage(content='Hi Tam! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8b761cb050', 'finish_reason': 'stop', 'logprobs': None}, id='run-a6e5560b-42f0-4835-b7d7-4eeca208fb06-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21}),
 HumanMessage(content="what's my name?", id='2c2a8960-99f2-40bd-99f1-323e44d235ae'),
 AIMessage(content='Your name is Tam! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 34, 'total_tokens': 46}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8b761cb050', 'finish_reason': 'stop', 'logprobs': None}, id='run-c4f3eed1-ab3e-439b-902c-bb44968b373d-0', usage_metadata={'input_tokens': 34, 'output_tokens': 12, 'total_tokens': 46})]

Remember, when deleting messages you will want to make sure that the remaining message list is still valid. This message list **may actually not be** - this is because it currently starts with an AI message, which some models do not allow.

In [13]:
input_message = HumanMessage(content="what's your day?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('ai', 'Hi Tam! How can I assist you today?'), ('human', "what's my name?"), ('ai', 'Your name is Tam! How can I help you today?'), ('human', "what's your day?")]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[('ai', 'Hi Tam! How can I assist you today?'), ('human', "what's my name?"), ('ai', 'Your name is Tam! How can I help you today?'), ('human', "what's your day?"), ('ai', "As an AI, I don't have feelings or days like humans do, but I'm here and ready to assist you! How can I help you today?")]
[('ai', 'Your name is Tam! How can I help you today?'), ('human', "what's your day?"), ('ai', "As an AI, I don't have feelings or days like humans do, but I'm here and ready to assist you! How can I help you today?")]
